Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

Note: Very good article that helps understanding 3d Convulution Networks Note: https://github.com/christianversloot/machine-learning-articles/blob/main/a-simple-conv3d-example-with-keras.md

**Check GPU and mount google drive**

In [ ]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-dccef7df-6ade-b039-257b-8ef6bf259508)


In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install scipy==1.1.0
#!pip install tensorflow==2.4.1
#pip install Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 31.2 MB 1.5 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc3 3.11.4 requires scipy>=1.2.0, but you have scipy 1.1.0 which is incompatible.
plotnine 0.6.0 requires scipy>=1.2.0, but you have scipy 1.1.0 which is incompatible.
jax 0.3.8 requires scipy>=1.2.1, but you have scipy 1.1.0 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [3]:
import numpy as np
import os
from scipy.misc import imread, imresize
from skimage.transform import resize
import datetime
import os
import warnings
warnings.filterwarnings('ignore')

from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D,Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers import SimpleRNN
from keras.layers import Dense, Activation
from keras.layers import LSTM,Dropout,GlobalAveragePooling2D,GlobalAveragePooling3D

We set the random seed so that the results don't vary drastically.

In [4]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
#tf.set_random_seed(30)
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the batch_size here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [5]:
train_doc = np.random.permutation(open('/content/drive/MyDrive/gesture_recognition/3D-CNN-Gesture-recognition/data_csv/20bn-jasper-subset-training-swipe-with-index.csv').readlines())
val_doc = np.random.permutation(open('/content/drive/MyDrive/gesture_recognition/3D-CNN-Gesture-recognition/data_csv/20bn-jasper-subset-validation-swipe-with-index.csv').readlines())
batch_size = 30
image_len=100;
image_wid=100;
print(val_doc[:60])

['94103;No gesture;0\n' '126501;No gesture;0\n' '57677;Swiping Down;3\n'
 '140963;No gesture;0\n' '74882;Swiping Up;4\n' '41155;Swiping Right;2\n'
 '130553;Swiping Up;4\n' '88550;Swiping Down;3\n' '95316;No gesture;0\n'
 '46455;Swiping Left;1\n' '102198;Swiping Down;3\n'
 '106416;Swiping Right;2\n' '119502;No gesture;0\n'
 '78620;Swiping Left;1\n' '3536;Swiping Right;2\n' '80053;No gesture;0\n'
 '95367;Swiping Down;3\n' '119719;Swiping Left;1\n' '85081;Swiping Up;4\n'
 '60742;No gesture;0\n' '43246;Swiping Left;1\n' '9539;No gesture;0\n'
 '129132;Swiping Up;4\n' '56082;Swiping Up;4\n' '121103;Swiping Right;2\n'
 '131907;No gesture;0\n' '5961;No gesture;0\n' '74741;Swiping Right;2\n'
 '65679;Swiping Up;4\n' '70777;Swiping Right;2\n'
 '25534;Swiping Right;2\n' '110127;Swiping Left;1\n'
 '20402;Swiping Up;4\n' '36520;Swiping Down;3\n' '119467;Swiping Down;3\n'
 '120178;Swiping Right;2\n' '56894;Swiping Up;4\n' '5787;No gesture;0\n'
 '111406;Swiping Left;1\n' '44606;Swiping Down;3\n'
 '110

# Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with img_idx, y,z and normalization such that you get high accuracy.

In [14]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [*range(0,30,2)]
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size;
#         print("Total batches:"+str(num_batches))
        for batch in range(num_batches): # we iterate over the number of batches
#             print("In batch"+str(batch))
            batch_data = np.zeros((batch_size,len(img_idx),image_len,image_wid,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
#                 print("index accessed:"+str(folder + (batch*batch_size)))
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0])# read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate over the frames/images of a folder to read them in

                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape
                    image = imresize(image,(image_len,image_wid))
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255 
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if len(t)%batch_size==0: continue
#         print("in part-2")
        batch_size2 = len(t) - batch_size*num_batches
#         print("batchSize2: "+str(batch_size2))
        batch_data = np.zeros((batch_size,len(img_idx),image_len,image_wid,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
        batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
        for folder in range(batch_size2): # iterate over the batch_size
#             print("index accessed:"+str(folder + (num_batches*batch_size)))
            imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
            for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape
                image = imresize(image,(image_len,image_wid))
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                batch_data[folder,idx,:,:,0] = image[:,:,0]/255
                batch_data[folder,idx,:,:,1] = image[:,:,1]/255
                batch_data[folder,idx,:,:,2] = image[:,:,2]/255
                    
            batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
        yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [ ]:
g = generator('/content/drive/MyDrive/gesture_recognition/data-set-full/extracted/Validation',val_doc,30)
print("-------------Iteration-1---------------")
next(g)
print("-------------Iteration-2---------------")
next(g)
print("-------------Iteration-3---------------")
next(g)
print("-------------Iteration-4---------------")
next(g)
print("-------------Iteration-5---------------")
next(g)
print("-------------Iteration-6---------------")
next(g)
print("-------------Iteration-7---------------")
next(g)

-------------Iteration-1---------------
Source path =  /content/drive/MyDrive/gesture_recognition/data-set-full/extracted/Validation ; batch size = 30
-------------Iteration-2---------------
-------------Iteration-3---------------
-------------Iteration-4---------------
-------------Iteration-5---------------
-------------Iteration-6---------------
-------------Iteration-7---------------


(array([[[[[0.1254902 , 0.17254902, 0.1254902 ],
           [0.10980392, 0.15686275, 0.10196078],
           [0.10980392, 0.16078431, 0.08235294],
           ...,
           [0.27843137, 0.35686275, 0.23921569],
           [0.27843137, 0.35686275, 0.25098039],
           [0.29019608, 0.36862745, 0.25490196]],
 
          [[0.10196078, 0.14901961, 0.10196078],
           [0.09803922, 0.14509804, 0.09803922],
           [0.10588235, 0.15686275, 0.09019608],
           ...,
           [0.28627451, 0.36078431, 0.2627451 ],
           [0.28235294, 0.35686275, 0.27058824],
           [0.29803922, 0.37254902, 0.28235294]],
 
          [[0.10588235, 0.14901961, 0.08627451],
           [0.10588235, 0.14901961, 0.09411765],
           [0.10588235, 0.14901961, 0.09411765],
           ...,
           [0.28627451, 0.36078431, 0.27843137],
           [0.27843137, 0.35294118, 0.27843137],
           [0.29411765, 0.36862745, 0.29411765]],
 
          ...,
 
          [[0.23921569, 0.17647059, 0.      

Hope it worked, next step is to verify

In [29]:
curr_dt_time = datetime.datetime.now()
train_path = '/content/drive/MyDrive/gesture_recognition/data-set-full/extracted/Train'
val_path = '/content/drive/MyDrive/gesture_recognition/data-set-full/extracted/Validation'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 250
print ('# epochs =', num_epochs)

# training sequences = 8928
# validation sequences = 1223
# epochs = 250


# Model
Here you make the model using different functionalities that Keras provides. Remember to use Conv3D and MaxPooling3D and not Conv2D and Maxpooling2D for a 3D convolution model. You would want to use TimeDistributed while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [34]:
#write your model here
# create architecture

# define parameters
n_output = 5 # number of classes in case of classification, 1 in case of regression
output_activation = 'softmax' # “softmax” or “sigmoid” in case of classification, “linear” in case of regression

model1 = Sequential()
model1.add(Conv3D(32,(2,2,2),activation='relu',padding='same',input_shape=(15,image_len,image_wid,3)))
model1.add(BatchNormalization())
model1.add(MaxPooling3D(pool_size=(2,2,2),padding='same'))
model1.add(Dropout(0.25))
model1.add(Conv3D(32,(2,2,2),activation='relu',padding='same'))
model1.add(BatchNormalization())
model1.add(MaxPooling3D(pool_size=(2,2,2),padding='same'))
model1.add(Dropout(0.25))
model1.add(Conv3D(64,(1,1,1),activation='relu',padding='same'))
model1.add(BatchNormalization())
model1.add(MaxPooling3D(pool_size=(1,2,1),padding='same'))
model1.add(Dropout(0.25))
model1.add(Conv3D(128,(3,3,3),activation='relu',padding='same'))
model1.add(BatchNormalization())
model1.add(MaxPooling3D(pool_size=(2,2,1),padding='same'))
model1.add(Dropout(0.25))
model1.add(GlobalAveragePooling3D())
model1.add(Dropout(0.25))
model1.add(Dense(512, activation = 'relu'))
model1.add(BatchNormalization())
model1.add(Dense(256, activation = 'relu'))
model1.add(BatchNormalization())
model1.add(Dense(n_output,activation=output_activation))

Now that you have written the model, the next step is to compile the model. When you print the summary of the model, you'll see the total number of parameters you have to train.

In [36]:
from tensorflow import keras
#from keras.optimizer_v2.adam import Adam as Adam

optimiser = keras.optimizers.Adam(lr=0.01)
model1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model1.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_4 (Conv3D)           (None, 15, 100, 100, 32)  800       
                                                                 
 batch_normalization_10 (Bat  (None, 15, 100, 100, 32)  128      
 chNormalization)                                                
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 8, 50, 50, 32)    0         
 3D)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 8, 50, 50, 32)     0         
                                                                 
 conv3d_5 (Conv3D)           (None, 8, 50, 50, 32)     8224      
                                                                 
 batch_normalization_11 (Bat  (None, 8, 50, 50, 32)   

Let us create the train_generator and the val_generator which will be used in .fit_generator.

In [16]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [25]:
saving_folder = "/content/drive/MyDrive/gesture_recognition/NeuralNets_3DCNN_Models"

model_name1 = 'model_init_1' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
model_name2 = 'model_init_2' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/' 
model_name3 = 'model_init_3' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'    
if not os.path.exists(model_name1):
    os.mkdir(model_name1)
if not os.path.exists(model_name2):
    os.mkdir(model_name2)
if not os.path.exists(model_name3):
    os.mkdir(model_name3)
        
#filepath1 = model_name1 + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_accuracy:.5f}.h5'
#filepath2 = model_name2 + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_accuracy:.5f}.h5'
#filepath3 = model_name3 + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_accuracy:.5f}.h5'

#edit: original formating
filepath1 = saving_folder + "/" + model_name1 + 'model-{epoch:05d}-{categorical_accuracy:.5f}.h5'
filepath2 = saving_folder + "/" + model_name2 + 'model-{epoch:05d}-{categorical_accuracy:.5f}.h5'
filepath3 = saving_folder + "/" + model_name3 + 'model-{epoch:05d}-{categorical_accuracy:.5f}.h5'

#replaced period with save_frequ because of depraction
checkpoint1 = ModelCheckpoint(filepath1, monitor='val_accuracy', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq=1)
checkpoint2 = ModelCheckpoint(filepath2, monitor='val_accuracy', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq=1)
checkpoint3 = ModelCheckpoint(filepath3, monitor='val_accuracy', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq=1)

LR =  ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)
callbacks_list1 = [checkpoint1, LR]
callbacks_list2 = [checkpoint2, LR]
callbacks_list3 = [checkpoint3, LR]

The steps_per_epoch and validation_steps are used by fit_generator to decide the number of next() calls it need to make.

In [26]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = int(num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = int(num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
model1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list1, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0) 

Epoch 1/250

Epoch 1: saving model to /content/drive/MyDrive/gesture_recognition/NeuralNets_3DCNN_Models/model_init_1_2022-06-1414_26_03.574001/model-00001-0.23333.h5
  1/298 [..............................] - ETA: 1:27:27 - loss: 2.2207 - categorical_accuracy: 0.2333
Epoch 1: saving model to /content/drive/MyDrive/gesture_recognition/NeuralNets_3DCNN_Models/model_init_1_2022-06-1414_26_03.574001/model-00001-0.26667.h5
  2/298 [..............................] - ETA: 11:54:12 - loss: 2.3928 - categorical_accuracy: 0.2667

# ALTERNATIVE MODELS

MODEL-2 (CNN+RNN)

In [17]:
#@title
from tensorflow import keras
from keras.applications.vgg16 import VGG16
from tensorflow.keras import Input

n_input=  (image_len,image_wid,3)
n_output = 5 # number of classes in case of classification, 1 in case of regression
output_activation = 'softmax'


cnn = VGG16(include_top=False,input_shape=n_input)

for layer in cnn.layers:
    layer.trainable=False
    
flat1= Flatten()(cnn.layers[-1].output)
cnn = Model(inputs=cnn.inputs, outputs=flat1)
model2= Sequential()
model2.add(TimeDistributed(cnn,input_shape=(15,image_len,image_wid,3)))
model2.add(LSTM(128, dropout=0.25,return_sequences=True))
model2.add(LSTM(128, dropout=0.25))
model2.add(Dense(64, activation = "relu"))
model2.add(BatchNormalization())
model2.add(Dense(5, activation = "softmax"))
optimiser = keras.optimizers.Adam(lr=0.01)
model2.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model2.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_3 (TimeDis  (None, 15, 4608)         14714688  
 tributed)                                                       
                                                                 
 lstm_6 (LSTM)               (None, 15, 128)           2425344   
                                                                 
 lstm_7 (LSTM)               (None, 128)               131584    
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 batch_normalization_3 (Batc  (None, 64)               256       
 hNormalization)                                                 
                                                                 
 dense_7 (Dense)             (None, 5)                

In [18]:
model2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list2, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/drive/MyDrive/gesture_recognition/data-set-full/extracted/Train ; batch size = 30
Epoch 1/30

Epoch 1: saving model to model_init_2_2022-06-1413_41_31.671701/model-00001.h5
  1/298 [..............................] - ETA: 1:26:47 - loss: 1.8888 - categorical_accuracy: 0.3000
Epoch 1: saving model to model_init_2_2022-06-1413_41_31.671701/model-00001.h5
  2/298 [..............................] - ETA: 11:23:53 - loss: 1.8644 - categorical_accuracy: 0.2333

KeyboardInterrupt: ignored

MODEL 3 (Modified Conv 3d)

In [ ]:
#@title
#write your model here
# create architecture

# define parameters
n_output = 5 # number of classes in case of classification, 1 in case of regression
output_activation = 'softmax' # “softmax” or “sigmoid” in case of classification, “linear” in case of regression

model3 = Sequential()
model3.add(Conv3D(32,(5,5,5),activation='relu',padding='same',input_shape=(15,image_len,image_wid,3)))
model3.add(BatchNormalization())
model3.add(MaxPooling3D(pool_size=(2,2,2)))
model3.add(Dropout(0.25))

model3.add(Conv3D(32,(3,3,3),activation='relu',padding='same'))
model3.add(BatchNormalization())
model3.add(MaxPooling3D(pool_size=(2,2,2)))
model3.add(Dropout(0.25))

model3.add(Conv3D(64,(2,2,2),activation='relu',padding='same'))

model3.add(Conv3D(128,(1,1,1),activation='relu',padding='same'))
model3.add(BatchNormalization())
model3.add(Dropout(0.25))

model3.add(Conv3D(128,(3,3,3),activation='relu',padding='same'))
model3.add(BatchNormalization())
model3.add(MaxPooling3D(pool_size=(2,2,1)))
model3.add(Dropout(0.25))

model3.add(GlobalAveragePooling3D())
model3.add(Dropout(0.25))
model3.add(Dense(512, activation = 'relu'))
model3.add(BatchNormalization())
model3.add(Dense(256, activation = 'relu'))
model3.add(BatchNormalization())
model3.add(Dense(5,activation=output_activation))
optimiser = optimizers.SGD(lr=0.01)
model3.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model3.summary())